In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 载入数据集
mnist = input_data.read_data_sets('./', one_hot=True)


# 设置每个 batch_size
batch_size = 100
# 共需多少批次运算
batch_nums = mnist.train.num_examples // batch_size


# 初始化权重
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# 初始化偏置
def biases_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


# 卷积层
def conv2d(x, w):
    """
    x: input tensor of shape [batch, in_height, in_width, in_channels]
    w: filter/kernel tensor of shape  [filter_height, filter_width, in_channels, out_channels]
    strides: stride[0] = strides[3] = 1, strides[1]: x方向的步长， strides[2]: 代表y方向的步长
    padding: 填充/补位 0, 'SAME':  , 'VALID':
    """
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')

# 池化层
def max_pool_2x2(x):
    # ksize=[1, x, y, 1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


# 定义 placeholder
# 28*28
x = tf.placeholder(tf.float32, [None, 784], name='x-input')
y = tf.placeholder(tf.float32, [None, 10], name='y-input')

# 改变x的格式为4d的向量 [batch, in_height, in_width, in_channels]
x_image = tf.reshape(x, shape=[-1, 28, 28, 1])

# 初始化第一个卷积的 权重 与 偏置
# 5*5 32个卷积核 1通道
w_conv1 = weight_variable([5, 5, 1, 32])
# 32个卷积核，32个偏置
b_conv1 = biases_variable([32])

# 第一层卷积运算
h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
# 池化操作
h_pool1 = max_pool_2x2(h_conv1)


# 初始化 第二层 卷积的 权重 与 偏置
# 5*5 ，64个卷积核， 32个：上一层的输出层个数
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = biases_variable([64])

# 第二层 卷积操作
h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#################################################################
##  28*28图片 第一次卷积操作后还是 28*28，第一次池化后 14*14  
##  第二次 卷积操作后 14*14， 第二次池化后 7*7               
##  经过上述操作后 最终得到 64张7*7图片                  
##################################################################


# 初始化第一个全连接 权重 与 偏置
# 上一层有 7*7*64个神经元，全连接层有 1024个神经元
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = biases_variable([1024])

# 把池化层2的输出 扁平化为 1维
h_pool2_flat = tf.reshape(h_pool2, shape=[-1, 7*7*64])
# 第一个全连接层 输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout操作
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)


# 初始化 第二个全连接层 权重 与 偏置
w_fc2 = weight_variable([1024, 10])
b_fc2 = biases_variable([10])

# 计算输出 
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# 损失函数 交叉熵
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))

# 优化器 AdamOptimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 结果存储在 一个 列表中
correct_predict = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

# 计算准确率
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))

# 变量初始化
init_op = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init_op)
    # 迭代
    for epoch in range(20):
        for batch in range(batch_nums):  
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 0.7})
            
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 0.7})
        print('Iter epoch {0}, Testing accuracy is {1}'.format(epoch, acc))




/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Iter epoch 0, Testing accuracy is 0.8511999845504761
Iter epoch 1, Testing accuracy is 0.8730000257492065
Iter epoch 2, Testing accuracy is 0.9739999771118164
Iter epoch 3, Testing accuracy is 0.9768999814987183
Iter epoch 4, Testing accuracy is 0.9801999926567078
Iter epoch 5, Testing accuracy is 0.9830999970436096
Iter epoch 6, Testing accuracy is 0.9812999963760376
Iter epoch 7, Testing accuracy is 0.9836000204086304


KeyboardInterrupt: 